In [8]:
#Modelo BETO
#Libreria transformers (modelo BERT predefinido para la clasificación (BertForSequenceClassification))
#Libreria sera BERT + Capa de clasificación por encima
#Debemos tokenizar nuestro dataset (tokens + attention mask + max_length)

import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, SequentialSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
import torch.optim
import numpy as np
import pandas as pd
import time
import datetime
import random
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from bs4 import BeautifulSoup
import torch
import numpy as np
from torch.utils.data import DataLoader
import pandas as pd
from textwrap import wrap
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from wordcloud import STOPWORDS
from bs4 import BeautifulSoup
import re,string 
from sklearn.metrics import classification_report,confusion_matrix
from string import punctuation
from nltk.corpus import wordnet
import torch
import os

import re, string

os.environ["TOKENIZERS_PARALLELISM"] = "false"

MAX_LEN = 32

# Select cpu or cuda
run_on = 'cpu'
device = torch.device(run_on)

df_train = pd.read_csv('/Users/nfanlo/dev/spanish-classifier-tfg/dataset/60-20-20/train.csv')
print(df_train.shape)
df_train.isnull().sum()
df_train.sentiment.replace("P" , 2 , inplace = True)
df_train.sentiment.replace("N" , 0 , inplace = True)
df_train.sentiment.replace("NEU" , 1, inplace = True)
df_train.head()
df_train['review'] = df_train['text']
df_train.drop('text', axis=1, inplace=True)
df_train['label'] = df_train['sentiment']
df_train.drop('sentiment', axis=1, inplace=True)

df_dev = pd.read_csv('/Users/nfanlo/dev/spanish-classifier-tfg/dataset/60-20-20/dev.csv')
print(df_dev.shape)
df_dev.isnull().sum()
df_dev.sentiment.replace("P" , 2 , inplace = True)
df_dev.sentiment.replace("N" , 0 , inplace = True)
df_dev.sentiment.replace("NEU" , 1, inplace = True)
df_dev['review'] = df_dev['text']
df_dev.drop('text', axis=1, inplace=True)
df_dev['label'] = df_dev['sentiment']
df_dev.drop('sentiment', axis=1, inplace=True)

(4340, 2)
(1447, 2)


In [9]:
print(df_train)

                                                 review  label
0     @Jorge_LBU Espero que te mejores pronto, vomit...      0
1     @ClubPaniniMx @PaniniComicsMx genial espero qu...      1
2                     kali uchis va a estar en bahidorá      1
3     @ferdiazgil @FranDguez sí, lo he leído ese tem...      0
4     @AbrahamMateoESP Exacto! Todo lo que él hace l...      1
...                                                 ...    ...
4335               tu en serio por que sois tan tontos       0
4336  Yo muchísimo mas . Hablamos al DM y te paso él...      1
4337  Les platicó algo bonito? Bueno pues, el último...      2
4338           Una breve reseña de quien soy realmente.      1
4339                   @MicaGiudice el tuyo quedo mejor      2

[4340 rows x 2 columns]


In [10]:
##CUSTOM DEFINED FUNCTIONS TO CLEAN THE TWEETS


#Remove punctuations, links, mentions and \r\n new line characters
def strip_all_entities(text): 
    text = text.replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower() #remove \n and \r and lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #remove links and mentions
    text = re.sub(r'[^\x00-\x7f]',r'', text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    banned_list= string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    return text

#clean hashtags at the end of the sentence, and keep those in the middle of the sentence by removing just the # symbol
def clean_hashtags(tweet):
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet)) #remove last hashtags
    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet)) #remove hashtags symbol from words in the middle of the sentence
    return new_tweet2

#Filter special characters such as & and $ present in some words
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

def remove_mult_spaces(text): # remove multiple spaces
    return re.sub("\s\s+" , " ", text)

In [11]:
review_new_train = []
review_new_dev = []
for t in df_train.review:
    review_new_train.append(remove_mult_spaces(filter_chars(clean_hashtags(strip_all_entities(t)))))

for t in df_dev.review:
    review_new_dev.append(remove_mult_spaces(filter_chars(clean_hashtags(strip_all_entities(t)))))

In [12]:
df_train['review'] = review_new_train
df_dev['review'] = review_new_dev

In [13]:
print(df_train)
print(df_dev)

                                                 review  label
0     espero que te mejores pronto vomitar es muy de...      0
1     genial espero que algun dia sacan uno de bleac...      1
2                      kali uchis va a estar en bahidor      1
3     s lo he ledo ese templo grande es el que he pu...      0
4     exacto todo lo que l hace lo hace con el coraz...      1
...                                                 ...    ...
4335                tu en serio por que sois tan tontos      0
4336  yo muchsimo mas hablamos al dm y te paso l apa...      1
4337  les platic algo bonito bueno pues el ltimo da ...      2
4338             una breve resea de quien soy realmente      1
4339                                el tuyo quedo mejor      2

[4340 rows x 2 columns]
                                                 review  label
0     ola wapa k tal stas spero k no seas una sorra ...      0
1                             tu optimismo es admirable      2
2                     que cora

In [19]:
df_train.review

0       espero que te mejores pronto vomitar es muy de...
1       genial espero que algun dia sacan uno de bleac...
2                        kali uchis va a estar en bahidor
3       s lo he ledo ese templo grande es el que he pu...
4       exacto todo lo que l hace lo hace con el coraz...
                              ...                        
4335                  tu en serio por que sois tan tontos
4336    yo muchsimo mas hablamos al dm y te paso l apa...
4337    les platic algo bonito bueno pues el ltimo da ...
4338               una breve resea de quien soy realmente
4339                                  el tuyo quedo mejor
Name: review, Length: 4340, dtype: object

In [10]:
review_new

['espero que te mejores pronto vomitar es muy desagradable',
 'genial espero que algun dia sacan uno de bleach y fairy tail',
 'kali uchis va a estar en bahidor',
 's lo he ledo ese templo grande es el que he puesto en ig ha perdido su parte superior y est inaccesible',
 'exacto todo lo que l hace lo hace con el corazn y desde el corazn',
 'soy de esos que an cuando la comida est lista y tenga hambre si no me sirven me da pereza y no como',
 'dios me ha sorprendido en esta semanauna caja de cafecosas de la tesisvienen cosas mejores ya sige lo de mi maestray una ellajajaja',
 'hizo un tiempazo en una ruta desafiante felicidades es muy muy campeona',
 'jaja yo llor cuando se muere mara jajaja cuando le lleva serenata es lo ms hermoso de la vidagtgtgtgtgtgt',
 'por rockear como loco ayer qued con la garganta mal y la nuca adolorida a pesar de ello el metal si q me logr desestrezar mucho',
 'por qu tiene que ser mi pelo tan goals mojado y tan mierda despues',
 'y faltaria conocer la config

In [11]:
df_train = pd.read_csv('/Users/nfanlo/dev/spanish-classifier-tfg/dataset/60-20-20/train.csv')
print(df_train.shape)
df_train.isnull().sum()
df_train.sentiment.replace("P" , 2 , inplace = True)
df_train.sentiment.replace("N" , 0 , inplace = True)
df_train.sentiment.replace("NEU" , 1, inplace = True)
df_train.head()
df_train['review'] = df_train['text']
df_train.drop('text', axis=1, inplace=True)
df_train['label'] = df_train['sentiment']
df_train.drop('sentiment', axis=1, inplace=True)

(4340, 2)


In [12]:
#Environment stopwords for train
stop = set(stopwords.words('spanish'))
punctuation = list(string.punctuation)
stop.update(punctuation)

#Data cleaning stopwords (ignored)
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop and i.strip().lower().isalpha():
            final_text.append(i.strip().lower())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text
#Apply function on review column
df_train['review']=df_train['review'].apply(denoise_text)

/Users/nfanlo/Library/Python/3.8/lib/python/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [13]:
df_train.review

0                     espero mejores vomitar desagradable
1         genial espero algun dia sacan bleach fairy tail
2                                  kali uchis va bahidorá
3       leído templo grande puesto perdido parte super...
4                               hace hace corazón corazón
                              ...                        
4335                                     serio tan tontos
4336       muchísimo mas hablamos dm paso apartado postal
4337      platicó bueno último día clases semestre regalo
4338                                         breve reseña
4339                                          quedo mejor
Name: review, Length: 4340, dtype: object